<a href="https://colab.research.google.com/github/dvashtom/colab-simulation-project/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#F
from __future__ import annotations
from typing import Dict, List, Optional

class Student:
    """
    Student entity.
    Fields:
        student_id: str
        name: str
        year: int
        completed_courses: Dict[str, float]  # course_id -> grade
        current_enrollments: List[Course]
    """
    def __init__(self, student_id: str, name: str, year: int) -> None:
        self.student_id = student_id
        self.name = name
        self.year = year
        self.completed_courses: Dict[str, float] = {}
        self.current_enrollments: List["Course"] = []

    def add_completed_course(self, course_id: str, grade: float) -> None:
        if not (0 <= grade <= 100):
            raise ValueError("grade must be between 0 and 100")
        self.completed_courses[course_id] = grade

    def remove_course(self, course: "Course") -> bool:
        """
        Remove a course from current_enrollments.
        Returns True if course was in the list and removed, else False.
        """
        if course in self.current_enrollments:
            self.current_enrollments.remove(course)
            return True
        return False

    def calculate_gpa(self) -> float:
        """
        Simple average over completed grades (0-100).
        You can convert to a 0-4 scale if desired.
        """
        if not self.completed_courses:
            return 0.0
        return sum(self.completed_courses.values()) / len(self.completed_courses)

    def grades_report(self) -> str:
        lines = [f"Grades report for {self.name} ({self.student_id})"]
        if not self.completed_courses:
            lines.append("No completed courses yet.")
        else:
            for cid, grade in self.completed_courses.items():
                lines.append(f"- {cid}: {grade}")
            lines.append(f"Average: {self.calculate_gpa():.2f}")
        return "\n".join(lines)

    def meeting_request(self, course: "Course", time_slot: str) -> str:
        """
        Create a simple meeting request message.
        """
        return f"Student {self.name} requests office hours for course {course.course_id} at {time_slot}."


# New Section

In [2]:

from __future__ import annotations
from typing import Dict, List, Optional
from datetime import date


class Exam:
    """
    Represents an exam in a specific course.
    Stores questions, max points, student scores and provides statistics.
    No 'finalize' method so external systems can modify grades (e.g., appeals).
    """

    def __init__(self, exam_id: str, course_id: str, exam_date: date):
        self.exam_id = exam_id
        self.course_id = course_id
        self.exam_date = exam_date

        # question_id -> max_points
        self.questions: Dict[str, float] = {}

        # student_id -> dict(question_id -> awarded_points)
        self.scores: Dict[str, Dict[str, float]] = {}

    # -------------------------
    # Question Management
    # -------------------------
    def add_question(self, question_id: str, max_points: float) -> None:
        if max_points <= 0:
            raise ValueError("max_points must be positive")
        if question_id in self.questions:
            raise ValueError("question already exists")
        self.questions[question_id] = max_points

    def total_max_points(self) -> float:
        return sum(self.questions.values())

    # -------------------------
    # Grading
    # -------------------------
    def set_student_score(self, student_id: str, question_id: str, points: float) -> None:
        if question_id not in self.questions:
            raise KeyError(f"Unknown question: {question_id}")
        if not (0 <= points <= self.questions[question_id]):
            raise ValueError(f"Invalid points for {question_id}")

        if student_id not in self.scores:
            self.scores[student_id] = {}

        self.scores[student_id][question_id] = points

    def student_total(self, student_id: str) -> float:
        if student_id not in self.scores:
            return 0.0
        return sum(self.scores[student_id].values())

    def student_percent(self, student_id: str) -> float:
        total = self.student_total(student_id)
        max_pts = self.total_max_points()
        if max_pts == 0:
            return 0.0
        return total / max_pts * 100.0

    def letter_grade(self, student_id: str) -> str:
        p = self.student_percent(student_id)
        if p >= 90: return "A"
        if p >= 80: return "B"
        if p >= 70: return "C"
        if p >= 60: return "D"
        return "F"

    # -------------------------
    # Statistics
    # -------------------------
    def statistics(self) -> Dict[str, float]:
        """Returns avg, min, max based on total numeric scores."""
        if not self.scores:
            return {"avg": 0.0, "min": 0.0, "max": 0.0}

        totals = [self.student_total(sid) for sid in self.scores]
        return {
            "avg": sum(totals) / len(totals),
            "min": min(totals),
            "max": max(totals),
        }


In [3]:

from __future__ import annotations
from typing import List, Optional

class Course:
    """
    Course entity.
    Fields:
        course_id: str
        name: str
        credits: int
        capacity: int
        prerequisites: List[str]
        lecturer: Optional[Lecturer]
        schedule_slot: Optional[str]  # e.g., "Mon 10:00"
        enrolled_students: List[str]  # list of student IDs
    """

    def __init__(
        self,
        course_id: str,
        name: str,
        credits: int,
        capacity: int,
        prerequisites: Optional[List[str]] = None,
        lecturer: Optional["Lecturer"] = None,
        schedule_slot: Optional[str] = None,
    ) -> None:
        if credits < 0:
            raise ValueError("credits must be non-negative")
        if capacity < 0:
            raise ValueError("capacity must be non-negative")

        self.course_id = course_id
        self.name = name
        self.credits = credits
        self.capacity = capacity
        self.prerequisites = list(prerequisites or [])
        self.lecturer = lecturer
        self.schedule_slot = schedule_slot
        self.enrolled_students: List[str] = []

    # Methods
    def assign_lecturer(self, lecturer: "Lecturer") -> None:
        self.lecturer = lecturer
        lecturer.assign_course(self)

    def set_schedule(self, schedule_slot: str) -> None:
        self.schedule_slot = schedule_slot

    def has_capacity(self) -> bool:
        return len(self.enrolled_students) < self.capacity

    def meets_prerequisites(self, student: "Student") -> bool:
        return all(req in student.completed_courses for req in self.prerequisites)

    def enroll(self, student: "Student") -> bool:
        """
        Enroll a student if:
          - not already enrolled (prevent duplicates)
          - capacity is available
          - prerequisites are met
          - no schedule conflict for the student
        Returns True if enrolled, False otherwise.
        """
        # Check for duplicate enrollment
        if student.student_id in self.enrolled_students:
            return False

        # Capacity check
        if not self.has_capacity():
            return False

        # Prerequisites
        if not self.meets_prerequisites(student):
            return False

        # Schedule conflicts (simple: compare same slot string)
        if self.schedule_slot and any(
            getattr(c, "schedule_slot", None) == self.schedule_slot for c in student.current_enrollments
        ):
            return False

        # All good
        self.enrolled_students.append(student.student_id)
        student.current_enrollments.append(self)
        return True

    def drop(self, student: "Student") -> bool:
        """
        Drop a student.
        Removes student from course.enrolled_students and removes course from student.current_enrollments.
        Returns True if was enrolled and removed, else False.
        """
        if student.student_id in self.enrolled_students:
            self.enrolled_students.remove(student.student_id)
            if self in student.current_enrollments:
                student.current_enrollments.remove(self)
            return True
        return False

    # Utility (not required but handy)
    def __repr__(self) -> str:
        return f"Course({self.course_id}, {self.name}, cap={self.capacity}, enrolled={len(self.enrolled_students)})"


In [4]:

from __future__ import annotations
from typing import List, Optional, Dict

class Lecturer:
    """
    Lecturer entity.
    Fields:
        lecturer_id: str
        name: str
        assigned_courses: List[Course]
    """
    def __init__(self, lecturer_id: str, name: str) -> None:
        self.lecturer_id = lecturer_id
        self.name = name
        self.assigned_courses: List["Course"] = []

    def assign_course(self, course: "Course") -> None:
        if course not in self.assigned_courses:
            self.assigned_courses.append(course)

    def remove_course(self, course_id: str) -> bool:
        for c in list(self.assigned_courses):
            if c.course_id == course_id:
                self.assigned_courses.remove(c)
                return True
        return False

    def is_available(self, slot: str) -> bool:
        """
        Simple availability check: lecturer is unavailable if any assigned course uses the same slot.
        """
        return all(getattr(c, "schedule_slot", None) != slot for c in self.assigned_courses)

    def submit_final_grade(self, course: "Course", exam_grades: Dict[str, float], homework_grades: Dict[str, float]) -> Dict[str, float]:
        """
        Calculate final grade for each enrolled student.
        Final grade = 80% exam grade + 20% homework grade.

        Args:
            course: The course object
            exam_grades: Dict mapping student_id to exam grade (0-100)
            homework_grades: Dict mapping student_id to homework grade (0-100)

        Returns:
            Dict mapping student_id to final grade (0-100)
        """
        result: Dict[str, float] = {}
        for sid in course.enrolled_students:
            exam = exam_grades.get(sid, 0.0)
            homework = homework_grades.get(sid, 0.0)
            final = (exam * 0.8) + (homework * 0.2)
            result[sid] = final
        return result

    def analyze_teaching_statistics(self, final_grades_per_course: Optional[Dict[str, Dict[str, float]]] = None) -> dict:
        """
        Return a dict with teaching statistics including:
        - lecturer_id and name
        - num_courses: number of assigned courses
        - avg_enrollment_per_course: average enrollment across courses
        - courses: list of course details with enrollment/capacity % and final grade average

        Args:
            final_grades_per_course: Optional dict mapping course_id to Dict[student_id -> final_grade]
        """
        stats = {
            "lecturer_id": self.lecturer_id,
            "lecturer_name": self.name,
            "num_courses": len(self.assigned_courses),
            "avg_enrollment_per_course": (
                sum(len(getattr(c, "enrolled_students", [])) for c in self.assigned_courses) / len(self.assigned_courses)
                if self.assigned_courses else 0.0
            ),
            "courses": [],
        }

        # Add per-course statistics
        for course in self.assigned_courses:
            enrolled = len(getattr(course, "enrolled_students", []))
            capacity = getattr(course, "capacity", 1)
            enrollment_percentage = (enrolled / capacity * 100) if capacity > 0 else 0.0

            # Calculate average final grade
            final_grade_average = 0.0
            if final_grades_per_course and getattr(course, "course_id", None) in final_grades_per_course:
                grades = final_grades_per_course[getattr(course, "course_id")]
                if grades:
                    final_grade_average = sum(grades.values()) / len(grades)

            course_stat = {
                "course_id": getattr(course, "course_id", "N/A"),
                "course_name": getattr(course, "name", "N/A"),
                "enrollment": enrolled,
                "capacity": capacity,
                "enrollment_percentage": enrollment_percentage,
                "final_grade_average": final_grade_average,
            }
            stats["courses"].append(course_stat)

        return stats





In [5]:


def test_add_student_lecturer_course_storage():
    dep = Department("CS")
    s = Student("S100", "Stu", 2)
    l = Lecturer("L100", "Dr X")
    c = Course("C100", "Intro", 3, 10)

    dep.add_student(s)
    dep.add_lecturer(l)
    dep.add_course(c)

    assert "S100" in dep.students
    assert dep.students["S100"] is s

    assert "L100" in dep.lecturers
    assert dep.lecturers["L100"] is l

    assert "C100" in dep.courses
    assert dep.courses["C100"] is c


def test_generate_schedule_report_conflicts_and_tbd():
    dep = Department("EE")
    c1 = Course("A1", "Alpha", 3, 5, schedule_slot="Mon 09:00")
    c2 = Course("B1", "Beta", 3, 5, schedule_slot="Mon 09:00")
    c3 = Course("C1", "Gamma", 3, 5)  # no schedule -> TBD

    dep.add_course(c1)
    dep.add_course(c2)
    dep.add_course(c3)

    report = dep.generate_schedule_report()

    # header
    assert "Schedule Report for Department: EE" in report

    # conflict flagged for Mon 09:00
    assert "Mon 09:00" in report
    assert "CONFLICT" in report

    # TBD entry present for course with no schedule
    assert "@ TBD" in report


def test_search_courses_by_id_name_lecturer_and_credits():
    dep = Department("Math")

    l = Lecturer("L1", "Prof Alice")
    c1 = Course("M101", "Calculus I", 4, 30, lecturer=l)
    c2 = Course("M201", "Linear Algebra", 3, 30)
    c3 = Course("STAT", "Statistics", 3, 30)

    dep.add_course(c1)
    dep.add_course(c2)
    dep.add_course(c3)

    # search by id (case-insensitive)
    res = dep.search_courses("m101")
    assert c1 in res and len(res) == 1

    # search by name substring
    res = dep.search_courses("linear")
    assert c2 in res and len(res) == 1

    # search by lecturer name
    res = dep.search_courses("alice")
    assert c1 in res and len(res) == 1

    # search by credits (digit string)
    res = dep.search_courses("3")
    # both c2 and c3 have 3 credits
    assert set(res) >= {c2, c3}


In [6]:

from __future__ import annotations
from typing import Dict, List, Optional

class Department:
    """
    Department aggregates students, lecturers and courses.
    Fields:
        name: str
        students: Dict[str, Student]
        lecturers: Dict[str, Lecturer]
        courses: Dict[str, Course]
    """
    def __init__(self, name: str) -> None:
        self.name = name
        self.students: Dict[str, "Student"] = {}
        self.lecturers: Dict[str, "Lecturer"] = {}
        self.courses: Dict[str, Course] = {}

    # Required methods
    def add_student(self, student: "Student") -> None:
        self.students[student.student_id] = student

    def add_lecturer(self, lecturer: "Lecturer") -> None:
        self.lecturers[lecturer.lecturer_id] = lecturer

    def add_course(self, course: Course) -> None:
        self.courses[course.course_id] = course

    def generate_schedule_report(self) -> str:
        """
        Creates a text schedule and flags time conflicts (simple same-slot conflicts).
        """
        lines: List[str] = [f"Schedule Report for Department: {self.name}"]
        slot_to_courses: Dict[str, List[str]] = {}
        for c in self.courses.values():
            slot = c.schedule_slot or "TBD"
            slot_to_courses.setdefault(slot, []).append(c.course_id)

        for slot, cids in slot_to_courses.items():
            if slot == "TBD":
                lines.append(f"- {', '.join(cids)} @ TBD")
            else:
                conflict = " (CONFLICT!)" if len(cids) > 1 else ""
                lines.append(f"- {slot}: {', '.join(cids)}{conflict}")
        return "\n".join(lines)

    def search_courses(self, search_term: str) -> List[Course]:
        """
        Search by term in id/name/lecturer name or match credits (if term is digit).
        """
        term = search_term.lower().strip()
        results: List[Course] = []
        for c in self.courses.values():
            by_id = term in c.course_id.lower()
            by_name = term in c.name.lower()
            by_lecturer = bool(c.lecturer and term in c.lecturer.name.lower())
            by_credits = term.isdigit() and int(term) == c.credits
            if by_id or by_name or by_lecturer or by_credits:
                results.append(c)
        return results


In [7]:


def test_enrollment_prereq_and_capacity():
    dep = Department("IE")
    l1 = Lecturer("L1", "Prof A")
    dep.add_lecturer(l1)
    c1 = Course("C1", "Intro", 3, 1, [], l1, "Mon 10:00")
    c2 = Course("C2", "Advanced", 3, 1, ["C1"], l1, "Mon 10:00")
    dep.add_course(c1); dep.add_course(c2)
    s = Student("S1", "Student One", 1)

    # cannot enroll without prereq
    assert c2.enroll(s) is False

    # take prereq
    assert c1.enroll(s) is True
    s.add_completed_course("C1", 85)

    # conflict (same slot) should block advanced even with prereq
    assert c2.enroll(s) is False


In [8]:

def test_student_add_completed_course_and_gpa_and_reports():
    s = Student("S10", "Bob", 2)

    # initial gpa 0 and report mentions no completed courses
    assert s.calculate_gpa() == 0.0
    report = s.grades_report()
    assert "No completed courses yet." in report

    # invalid grade raises
    try:
        s.add_completed_course("C1", -5)
        assert False, "expected ValueError for negative grade"
    except ValueError:
        pass

    try:
        s.add_completed_course("C1", 105)
        assert False, "expected ValueError for grade > 100"
    except ValueError:
        pass

    # add valid courses
    s.add_completed_course("C1", 80)
    s.add_completed_course("C2", 90)
    assert abs(s.calculate_gpa() - 85.0) < 1e-6
    rpt = s.grades_report()
    assert "C1" in rpt and "C2" in rpt and "Average:" in rpt

    # meeting request format
    dummy_course = Course("X", "Dummy", 1, 10)
    mr = s.meeting_request(dummy_course, "Fri 12:00")
    assert "requests office hours" in mr and "Fri 12:00" in mr


def test_lecturer_assign_remove_avail_and_stats_and_submit():
    lec = Lecturer("L10", "Dr Test")
    c1 = Course("C1", "One", 3, 5, schedule_slot="Mon 11:00")
    c2 = Course("C2", "Two", 3, 5, schedule_slot="Tue 12:00")

    # assign courses
    lec.assign_course(c1)
    assert c1 in lec.assigned_courses
    # duplicate assign should not duplicate
    lec.assign_course(c1)
    assert lec.assigned_courses.count(c1) == 1

    # remove course
    assert lec.remove_course("C1") is True
    assert c1 not in lec.assigned_courses
    # removing again returns False
    assert lec.remove_course("C1") is False

    # availability
    lec.assign_course(c1)
    assert lec.is_available("Tue 12:00") is True
    assert lec.is_available("Mon 11:00") is False

    # assign c2 as well for statistics
    lec.assign_course(c2)

    # submit_final_grade with weighted calculation: 80% exam + 20% homework
    c2.enrolled_students = ["sA", "sB"]
    exam_grades = {"sA": 90.0, "sB": 80.0}
    homework_grades = {"sA": 85.0, "sB": 95.0}
    grades = lec.submit_final_grade(c2, exam_grades, homework_grades)
    assert isinstance(grades, dict)
    assert set(grades.keys()) == {"sA", "sB"}
    # sA: (90 * 0.8) + (85 * 0.2) = 72 + 17 = 89.0
    assert abs(grades["sA"] - 89.0) < 0.01
    # sB: (80 * 0.8) + (95 * 0.2) = 64 + 19 = 83.0
    assert abs(grades["sB"] - 83.0) < 0.01

    # analyze_teaching_statistics with enrollment% and final grade average
    stats = lec.analyze_teaching_statistics()
    assert stats["lecturer_id"] == "L10"
    assert stats["num_courses"] == len(lec.assigned_courses)
    assert "courses" in stats
    assert len(stats["courses"]) >= 1
    # Check structure of course stats
    for course_stat in stats["courses"]:
        assert "course_id" in course_stat
        assert "enrollment_percentage" in course_stat
        assert "final_grade_average" in course_stat
        assert 0 <= course_stat["enrollment_percentage"] <= 100


def test_course_drop_with_student_and_student_remove_course():
    s = Student("S20", "Diana", 3)
    c = Course("C10", "Biology", 3, 10)

    # enroll
    assert c.enroll(s) is True
    assert s.student_id in c.enrolled_students
    assert c in s.current_enrollments

    # drop with Student object removes from both places
    assert c.drop(s) is True
    assert s.student_id not in c.enrolled_students
    assert c not in s.current_enrollments

    # drop when not enrolled returns False
    s2 = Student("S21", "Eve", 1)
    assert c.drop(s2) is False

    # Student.remove_course()
    s3 = Student("S22", "Frank", 2)
    c2 = Course("C11", "History", 2, 5)
    s3.current_enrollments.append(c2)
    assert c2 in s3.current_enrollments
    assert s3.remove_course(c2) is True
    assert c2 not in s3.current_enrollments

    # remove_course when not in list returns False
    c3 = Course("C12", "Art", 1, 5)
    assert s3.remove_course(c3) is False


In [9]:
class AppealsManager:
    def __init__(self):
        self.appeals = {}     # appeal_id → dict with all appeal info
        self.counter = 1      # for generating IDs

    def _generate_id(self):
        aid = f"A{self.counter:04d}"
        self.counter += 1
        return aid

    # ------------------------------------------------------
    # 1. הסטודנט מגיש ערעור
    # ------------------------------------------------------
    def submit_appeal(self, student, course, exam, reason, requested_grade=None):
        if exam.course_id != course.course_id:
            raise ValueError("המבחן לא שייך לקורס")

        if student.student_id not in exam.student_grades:
            raise ValueError("אין לסטודנט ציון במבחן הזה")

        appeal_id = self._generate_id()

        self.appeals[appeal_id] = {
            "id": appeal_id,
            "student": student,
            "course": course,
            "exam": exam,
            "reason": reason,
            "requested_grade": requested_grade,
            "status": "pending",
            "reviewer": None,
            "response_note": None
        }

        return appeal_id

    # ------------------------------------------------------
    # 2. המרצה מאשר / דוחה את הערעור
    # ------------------------------------------------------
    def review_appeal(self, appeal_id, lecturer, approve, note=None, new_grade=None):
        if appeal_id not in self.appeals:
            raise ValueError("ערעור לא נמצא")

        appeal = self.appeals[appeal_id]

        # רק המרצה של הקורס יכול לטפל בערעור
        if lecturer.lecturer_id != appeal["course"].lecturer.lecturer_id:
            raise PermissionError("מרצה זה לא מלמד את הקורס")

        if appeal["status"] != "pending":
            raise ValueError("הערעור כבר טופל")

        if approve:
            appeal["status"] = "approved"
            appeal["requested_grade"] = new_grade if new_grade is not None else appeal["requested_grade"]
        else:
            appeal["status"] = "rejected"

        appeal["reviewer"] = lecturer
        appeal["response_note"] = note

        return appeal

    # ------------------------------------------------------
    # 3. עדכון ציון לאחר אישור
    # ------------------------------------------------------
    def apply_grade_update(self, appeal_id):
        if appeal_id not in self.appeals:
            raise ValueError("ערעור לא נמצא")

        appeal = self.appeals[appeal_id]

        if appeal["status"] != "approved":
            return None

        new_grade = appeal["requested_grade"]
        if new_grade is None:
            return None

        student = appeal["student"]
        exam = appeal["exam"]
        course = appeal["course"]

        # עדכון הציון במבחן ובסטודנט
        exam.student_grades[student.student_id] = new_grade
        student.completed_courses[course.course_id] = new_grade

        return new_grade

    # ------------------------------------------------------
    # 4. דוח ערעורים
    # ------------------------------------------------------
    def report(self):
        report_list = []
        for a in self.appeals.values():
            report_list.append({
                "appeal_id": a["id"],
                "student": a["student"].name,
                "course": a["course"].name,
                "exam": a["exam"].exam_id,
                "status": a["status"],
                "requested_grade": a["requested_grade"],
                "reviewer": a["reviewer"].name if a["reviewer"] else None,
                "note": a["response_note"]
            })
        return report_list


In [10]:
from datetime import date

class SimpleExamForAppeal:
    """
    Minimal exam-like object used only for the AppealsManager demo.
    It matches the interface expected by AppealsManager: exam_id, course_id, student_grades.
    """
    def __init__(self, exam_id: str, course: "Course", initial_grades: dict):
        self.exam_id = exam_id
        self.course_id = course.course_id
        # student_id -> numeric grade
        self.student_grades = dict(initial_grades)


def main():
    """
    Single compact main function that:
    - Creates a small university department simulation
    - Demonstrates EVERY method of all classes:
      Student, Course, Lecturer, Department, Exam, AppealsManager
    - Keeps console output relatively short and readable
    """

    print("=" * 72)
    print("UNIVERSITY DEPARTMENT SYSTEM - SINGLE MAIN DEMO (ALL METHODS)")
    print("=" * 72)

    # ------------------------------------------------------------------
    # 1. Base entities: lecturer, students, courses, department
    # ------------------------------------------------------------------
    # Create one lecturer that will teach several courses
    lecturer = Lecturer("L001", "Dr. Smith")

    # Create two students in different years
    student_a = Student("S001", "Alice", 1)
    student_b = Student("S002", "Bob", 2)

    # Student B completes MATH101 so he can satisfy prerequisites later
    student_b.add_completed_course("MATH101", 82.0)

    # Create two courses with different schedules and prerequisites
    course_intro = Course(
        course_id="IE101",
        name="Intro to Industrial Engineering",
        credits=3,
        capacity=2,
        prerequisites=["MATH101"],
        schedule_slot="Mon 10:00-11:30",
    )
    course_stats = Course(
        course_id="IE201",
        name="Statistics for Engineers",
        credits=4,
        capacity=2,
        prerequisites=[],
        schedule_slot="Wed 10:00-11:30",
    )

    # Create a department representing Industrial Engineering & Management
    dept = Department("Industrial Engineering and Management")

    # ------------------------------------------------------------------
    # 2. COURSE methods (8): __init__, assign_lecturer, set_schedule,
    #    has_capacity, meets_prerequisites, enroll, drop, __repr__
    # ------------------------------------------------------------------
    print("\n[COURSE METHODS]")
    # Assign the lecturer to the intro course
    course_intro.assign_lecturer(lecturer)
    print(f"Assigned lecturer {lecturer.name} to course {course_intro.course_id}")

    # Change the schedule of the intro course
    course_intro.set_schedule("Tue 14:00-15:30")
    print(f"Updated schedule for {course_intro.course_id} to {course_intro.schedule_slot}")

    # Capacity check before and after enrollment
    print(f"Has capacity before enrollment: {course_intro.has_capacity()}")

    # Check prerequisites for each student
    print(f"Alice meets prerequisites: {course_intro.meets_prerequisites(student_a)}")
    print(f"Bob meets prerequisites:   {course_intro.meets_prerequisites(student_b)}")

    # Enroll Bob (valid case)
    enrolled_ok = course_intro.enroll(student_b)
    print(f"Enrollment of Bob to {course_intro.course_id} succeeded: {enrolled_ok}")
    print(f"Has capacity after Bob enrolled: {course_intro.has_capacity()}")

    # Try duplicate enrollment (should be prevented)
    duplicate_enrollment = course_intro.enroll(student_b)
    print(f"Duplicate enrollment prevented: {not duplicate_enrollment}")

    # Drop Bob from the course
    dropped_ok = course_intro.drop(student_b)
    print(f"Dropping Bob from {course_intro.course_id} succeeded: {dropped_ok}")

    # Show representation of course object
    print(f"Course __repr__: {repr(course_intro)}")

    # ------------------------------------------------------------------
    # 3. STUDENT methods (6):
    #    __init__, add_completed_course, calculate_gpa, grades_report,
    #    meeting_request, remove_course
    # ------------------------------------------------------------------
    print("\n[STUDENT METHODS]")

    # Add completed courses and grades for Alice
    student_a.add_completed_course("IE001", 90.0)
    student_a.add_completed_course("IE050", 80.0)
    student_a.add_completed_course("MATH101", 88.0)

    # Calculate and display GPA for Alice
    gpa_a = student_a.calculate_gpa()
    print(f"Alice GPA: {gpa_a:.2f}")

    # Display grades report for Alice
    print("Alice grades report:")
    print(student_a.grades_report())

    # Meeting request for Alice with the intro course
    meeting_msg = student_a.meeting_request(course_intro, "Thu 15:00")
    print("Meeting request message:")
    print(meeting_msg)

    # Demonstrate remove_course: enroll Alice to Statistics, then remove it
    course_stats.enroll(student_a)
    print(f"Current enrollments for Alice before remove_course: "
          f"{[c.course_id for c in student_a.current_enrollments]}")
    removed = student_a.remove_course(course_stats)
    print(f"student_a.remove_course(course_stats) returned: {removed}")
    print(f"Current enrollments for Alice after remove_course: "
          f"{[c.course_id for c in student_a.current_enrollments]}")

    # ------------------------------------------------------------------
    # 4. LECTURER methods (6):
    #    __init__, assign_course, remove_course, is_available,
    #    submit_final_grade, analyze_teaching_statistics
    # ------------------------------------------------------------------
    print("\n[LECTURER METHODS]")

    # Assign courses to lecturer (in addition to the one already assigned)
    lecturer.assign_course(course_intro)
    lecturer.assign_course(course_stats)
    print(f"Lecturer {lecturer.name} assigned courses: "
          f"{[c.course_id for c in lecturer.assigned_courses]}")

    # Remove one course by ID
    lecturer.remove_course("IE201")
    print(f"After removing IE201: {[c.course_id for c in lecturer.assigned_courses]}")

    # Check lecturer availability at two different time slots
    print(f"Is lecturer available at Tue 14:00? {lecturer.is_available('Tue 14:00-15:30')}")
    print(f"Is lecturer available at Mon 09:00? {lecturer.is_available('Mon 09:00-10:00')}")

    # Prepare dummy enrollment and grades for submit_final_grade
    course_intro.enrolled_students = [student_b.student_id]
    exam_grades = {student_b.student_id: 90.0}
    homework_grades = {student_b.student_id: 80.0}

    final_grades_intro = lecturer.submit_final_grade(course_intro, exam_grades, homework_grades)
    print(f"Final grades for {course_intro.course_id}: {final_grades_intro}")

    # Analyze teaching statistics using the computed final grades
    stats = lecturer.analyze_teaching_statistics(
        final_grades_per_course={course_intro.course_id: final_grades_intro}
    )
    print("Lecturer teaching statistics (compact view):")
    print(f"  Lecturer: {stats['lecturer_name']} ({stats['lecturer_id']})")
    print(f"  Number of courses: {stats['num_courses']}")
    print(f"  Average enrollment per course: {stats['avg_enrollment_per_course']:.1f}")

    # ------------------------------------------------------------------
    # 5. DEPARTMENT methods (6):
    #    __init__, add_student, add_lecturer, add_course,
    #    generate_schedule_report, search_courses
    # ------------------------------------------------------------------
    print("\n[DEPARTMENT METHODS]")

    # Add students, lecturer and courses to the department
    dept.add_student(student_a)
    dept.add_student(student_b)
    dept.add_lecturer(lecturer)
    dept.add_course(course_intro)
    dept.add_course(course_stats)

    print(f"Department '{dept.name}' now has "
          f"{len(dept.students)} students, {len(dept.lecturers)} lecturers, "
          f"{len(dept.courses)} courses.")

    # Generate schedule report (can include conflicts if any)
    schedule_report = dept.generate_schedule_report()
    print("\nDepartment schedule report (short):")
    print(schedule_report)

    # Search for courses containing 'IE1' in id/name
    found_courses = dept.search_courses("IE1")
    print("\nSearch results for 'IE1':")
    for c in found_courses:
        print(f"  - {c.course_id}: {c.name}")

    # ------------------------------------------------------------------
    # 6. EXAM methods (8):
    #    __init__, add_question, total_max_points,
    #    set_student_score, student_total, student_percent,
    #    letter_grade, statistics
    # ------------------------------------------------------------------
    print("\n[EXAM METHODS]")

    exam = Exam(exam_id="E101", course_id=course_intro.course_id, exam_date=date(2026, 1, 15))

    # Define three questions and their maximum points
    exam.add_question("Q1", 30.0)
    exam.add_question("Q2", 40.0)
    exam.add_question("Q3", 30.0)

    max_points = exam.total_max_points()
    print(f"Total maximum points in exam {exam.exam_id}: {max_points}")

    # Set scores for two students
    exam.set_student_score(student_a.student_id, "Q1", 24.0)
    exam.set_student_score(student_a.student_id, "Q2", 32.0)
    exam.set_student_score(student_a.student_id, "Q3", 27.0)

    exam.set_student_score(student_b.student_id, "Q1", 18.0)
    exam.set_student_score(student_b.student_id, "Q2", 28.0)
    exam.set_student_score(student_b.student_id, "Q3", 21.0)

    # Compute totals, percentages and letter grades
    print("Per-student exam results:")
    for s in (student_a, student_b):
        total = exam.student_total(s.student_id)
        percent = exam.student_percent(s.student_id)
        letter = exam.letter_grade(s.student_id)
        print(f"  {s.name}: total={total:.1f}, percent={percent:.1f}%, letter={letter}")

    # Overall exam statistics
    exam_stats = exam.statistics()
    print("Exam statistics:")
    print(f"  Average: {exam_stats['avg']:.1f}")
    print(f"  Min:     {exam_stats['min']:.1f}")
    print(f"  Max:     {exam_stats['max']:.1f}")

    # ------------------------------------------------------------------
    # 7. APPEALS MANAGER methods (6, including internal _generate_id):
    #    __init__, submit_appeal, review_appeal, apply_grade_update, report
    #    (_generate_id is used internally by submit_appeal)
    # ------------------------------------------------------------------
    print("\n[APPEALS MANAGER METHODS]")

    appeals_manager = AppealsManager()

    # We prepare a simple exam object compatible with AppealsManager
    # and an initial final grade for Bob in IE101
    student_b.completed_courses[course_intro.course_id] = 75.0
    simple_exam = SimpleExamForAppeal(
        exam_id="E-APPEAL",
        course=course_intro,
        initial_grades={student_b.student_id: 75.0},
    )

    # Student Bob submits an appeal
    appeal_id = appeals_manager.submit_appeal(
        student=student_b,
        course=course_intro,
        exam=simple_exam,
        reason="Re-check final calculation",
        requested_grade=90.0,
    )
    print(f"Appeal submitted with id {appeal_id}")

    # Lecturer reviews and approves the appeal with a new grade
    reviewed = appeals_manager.review_appeal(
        appeal_id=appeal_id,
        lecturer=lecturer,
        approve=True,
        note="Adjusted after manual review",
        new_grade=88.0,
    )
    print(f"Appeal status after review: {reviewed['status']}, "
          f"requested_grade={reviewed['requested_grade']}")

    # Apply the grade update to the exam and the student's record
    updated_grade = appeals_manager.apply_grade_update(appeal_id)
    print(f"Updated grade for Bob in {course_intro.course_id}: {updated_grade}")
    print(f"Student record now shows: "
          f"{student_b.completed_courses[course_intro.course_id]}")

    # Generate a compact appeals report
    appeals_report = appeals_manager.report()
    print("\nAppeals report:")
    for r in appeals_report:
        print(f"  Appeal {r['appeal_id']}: student={r['student']}, "
              f"course={r['course']}, status={r['status']}, "
              f"requested={r['requested_grade']}, reviewer={r['reviewer']}")

    # ------------------------------------------------------------------
    # Final summary line – all methods have been executed under one main
    # ------------------------------------------------------------------
    print("\n" + "=" * 72)
    print("ALL CLASS METHODS WERE EXECUTED SUCCESSFULLY IN A SINGLE MAIN()")
    print("=" * 72)


# In a Jupyter/Colab notebook, __name__ is '__main__', so this will run
# when you execute the cell. You can also call main() manually if needed.
if __name__ == "__main__":
    main()


UNIVERSITY DEPARTMENT SYSTEM - ALL METHODS DEMONSTRATION

[COURSE CLASS - 8 METHODS]

1. Course.__init__()
   Created: CS101 - Intro to Python (3 credits, 30 capacity)

2. Course.assign_lecturer()
   Assigned Dr. Johnson to CS101

3. Course.set_schedule()
   Updated schedule to Tue 14:00-15:30

4. Course.has_capacity()
   CS102 has capacity: True
   After 2 students: False (capacity full)

5. Course.meets_prerequisites()
   Alice meets prereq for CS101: True
   Bob meets prereq for CS101: False

6. Course.enroll()
   First enrollment: True
   Duplicate enrollment attempt: False (prevented)

7. Course.drop()
   Charlie dropped from CS103
   Students in CS103: []

8. Course.__repr__()
   Course(CS103, Algorithms, cap=30, enrolled=0)

[STUDENT CLASS - 6 METHODS]

1. Student.__init__()
   Created: Diana (ID: S004, Year: 3)

2. Student.add_completed_course()
   Diana's completed courses: {'CS101': 95, 'CS102': 87, 'MATH101': 92}

3. Student.calculate_gpa()
   Diana's GPA: 91.33

4. Student.